In [5]:
# Establish a connection between Python and the Sakila database
import pandas as pd
from sqlalchemy import create_engine

# Replace these values with your actual database credentials
user = 'root'
password = 'DCEghh20180817'
host = 'localhost'
database = 'sakila'

# Create the connection engine
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

In [6]:
# Write the rentals_month function
def rentals_month(engine, month, year):
    query = f"""
    SELECT customer_id, rental_date
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    return pd.read_sql(query, engine)

In [7]:
# Write the rental_count_month function
def rental_count_month(df, month, year):
    rental_counts = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_counts

In [8]:
# Write the compare_rentals function
def compare_rentals(df1, df2):
    combined = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2')).fillna(0)
    combined['difference'] = combined.iloc[:, 1] - combined.iloc[:, 2]
    return combined

In [10]:
# Retrieve rental data for May and June 2005
df_may = rentals_month(engine, 5, 2005)
df_june = rentals_month(engine, 6, 2005)

# Count rentals by customer for May and June 2005
rental_counts_may = rental_count_month(df_may, 5, 2005)
rental_counts_june = rental_count_month(df_june, 6, 2005)

# Compare the rental activity between May and June 2005
comparison = compare_rentals(rental_counts_may, rental_counts_june)

# Display the comparison
display(comparison)

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,5,3.0,5.0,-2.0
4,6,3.0,4.0,-1.0
...,...,...,...,...
593,583,0.0,6.0,-6.0
594,585,0.0,4.0,-4.0
595,591,0.0,3.0,-3.0
596,592,0.0,5.0,-5.0
